In [ ]:
TELEGRAM_BOT_TOKEN = ''
TELEGRAM_CHAT_ID = ''

In [ ]:
import gzip
import pandas as pd
import os
import tarfile 
import zipfile
import tensorflow as tf
import time
import string

In [ ]:
base_dir = '.'
dataset_folder = os.path.join(base_dir,'dataset_parsed')
dataset_new_folder = os.path.join(base_dir, 'dataset_parsed_new')

In [ ]:
def save_df_to_csv(df, filename):
  df.to_csv(os.path.join(base_dir, filename))

In [ ]:
def remove_punct(corpus_path):
    corpus_path, dirs, files = next(os.walk(corpus_path))
    
    for f in files:
        print(f"{time.strftime('%H:%M:%S')} Current file: {f}")
        words_removed = 0
        try:
            data_path = os.sep.join([corpus_path, f])
            document = pd.read_csv(data_path)
            document.dropna(subset=['text'])
            for index, row in document.iterrows():
                # print(row['text'])
                if isinstance(row['text'], str):
                    words_removed += len(row['text'].split(' '))
                    row['text'] = ' '.join([word for word in row['text'].split(' ') if not set(word).intersection(set(string.punctuation.replace('+', '').replace('-', '')))])
                    words_removed -= len(row['text'].split(' '))
                    document.at[index,'text'] = row['text']
            print(f"{time.strftime('%H:%M:%S')} saving {os.path.join(dataset_new_folder, f)}")
            save_df_to_csv(document, os.path.join(dataset_new_folder, f))
            print(f"{time.strftime('%H:%M:%S')} removed {words_removed} words")
        except Exception as e:
            print(e)

In [ ]:
remove_punct(dataset_folder)

In [ ]:
def send_telegram_message(text):
    if TELEGRAM_BOT_TOKEN and TELEGRAM_CHAT_ID:
        !pip install -q python-telegram-bot
        from telegram import Bot
        bot = Bot(token=TELEGRAM_BOT_TOKEN)
        bot.send_message(text=text, chat_id=TELEGRAM_CHAT_ID)
send_telegram_message("Finito di rimuovere punteggiatura da dataset completo")

In [ ]:
from collections import Counter
file_num = 0
words_set = set()
cnt = Counter()
def calculate_file_num(corpus_path):
    global file_num
    corpus_path, dirs, files = next(os.walk(corpus_path))
    
    for f in files:
        print(f)
        try:
            data_path = os.sep.join([corpus_path, f])
            document = pd.read_csv(data_path)
            document.dropna(subset=['text'])
            file_num += len(document['text'])
            for index, row in document.iterrows():
                # print(row['text'])
                if isinstance(row['text'], str):
                    for word in row['text'].split(' '):
                        words_set.add(word.lower())
                        cnt[word.lower()] += 1
            print(f"{time.strftime('%H:%M:%S')} numero testi: {len(document['text'])}")
            print(f"{time.strftime('%H:%M:%S')} lunghezza set parole: {len(words_set)}")
        except Exception as e:
            print(e)

In [ ]:
calculate_file_num("dataset_parsed_new")

In [ ]:
print(f"numero testi: {file_num}")
print(f"lunghezza set parole: {len(words_set)}")
print(cnt.most_common(50))
for x,y in cnt.most_common(50):
    print(f"{x}: {y}")

In [ ]:
send_telegram_message(f"Finito di salvare stat dataset:\nnumero testi: {file_num}\nlunghezza set parole: {len(words_set)}\n{cnt.most_common(10)}")